In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from os import listdir
from os.path import isfile, join
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.cluster import AgglomerativeClustering, KMeans, MiniBatchKMeans
from sklearn_extra.cluster import KMedoids
import numpy as np
import xgboost as xgb
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from numpy.random import random_sample
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import sklearn.metrics as metrics
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import sklearn

import itertools


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [3]:
# -*- coding: utf-8 -*-
"""Surrogate_Sv5.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1DxToJ3EEWQokxhO20e5k2OpR-_Nr5l_R

#Importando base e criando variáveis
"""


import wandb
import sys

### WANDB
api = wandb.Api()
###

#PATH = "/home/barbon/PycharmProjects/Surrogate_SV5/"
PATH = sys.argv[1]
VERSION = "Sv5_b"
FILE = "metadatabase_surrogate_"+VERSION+".csv"
LOG_FILE = "log_13_07_b.csv"

###################

par_ini = sys.argv[2]
par_fin = sys.argv[3]


sil_ini_list = np.arange(0.2, 0.6, 0.05)
dbs_max_list = np.arange(0.4, 0.8, 0.1)
dist_sil_dbs_list = np.arange(0.15, 0.35, 0.05)

seed_list = range(int(par_ini),int(par_fin))
distribution_list = [["exponential", 'gumbel', 'normal'], ["exponential", 'normal'], ['gumbel', 'normal']]
#distribution_list = [["exponential", 'normal']]
algorithms = [RandomForestRegressor]
qtd_arvores = [100,250]
threshold_qtd_on_training = 50
contamination = [0, 0.05, 0.1, 0.15]
###################








"""#Visualizando dados"""

import matplotlib.pyplot as plt

def datasets_statistics(meta_dataset):
  #meta_dataset = pd.read_csv(PATH+FILE)
  print("Original", meta_dataset.shape)

  #meta_dataset = meta_dataset.sample(frac=1, random_state=1, replace=True).reset_index(drop=True)
  #meta_dataset = meta_dataset.drop_duplicates(subset=["Dataset"], keep="last")
  #print("Final", meta_dataset.shape)

  meta_dataset['clusters'] = meta_dataset.apply(lambda row: int(row.Dataset.split('-')[1].replace("clusters","")), axis=1)
  meta_dataset['aspectref'] = meta_dataset.apply(lambda row: float(row.Dataset.split('-')[7].replace("aspectref","")), axis=1)
  meta_dataset['aspectmaxmin'] = meta_dataset.apply(lambda row: float(row.Dataset.split('-')[8].replace("aspectmaxmin","")), axis=1)
  meta_dataset['radius'] = meta_dataset.apply(lambda row: float(row.Dataset.split('-')[9].replace("radius","")), axis=1)
  meta_dataset['imbalance'] = meta_dataset.apply(lambda row: float(row.Dataset.split('-')[10].replace("imbalance","")), axis=1)
  meta_dataset['distribution'] = meta_dataset.apply(lambda row: row.Dataset.split('\'')[1], axis=1)

  plt.figure()
  meta_dataset["cluster_diff"].hist(bins=len(meta_dataset["cluster_diff"].unique()))
  plt.figure()
  meta_dataset["clusters"].hist()
  plt.figure()
  meta_dataset["algorithm"].hist()
  plt.figure()
  meta_dataset["distribution"].hist()
  plt.figure()
  meta_dataset["aspectref"].hist()
  plt.figure()
  meta_dataset["sil"].hist(color='#FFCF56')
  plt.figure()
  meta_dataset["dbs"].hist(color='#FF0000')

"""#Surrogate Model

### methods
"""

def filtering_distribution(database_input, dist):
  database_input['distribution'] = database_input.apply(lambda row: row.Dataset.split('\'')[1], axis=1)
  return database_input[database_input.distribution.isin(dist)].drop(["distribution"], axis=1)

def filter_sil_bds(database_input, min_sil, max_dbs):
  database_input = database_input[database_input.sil>=min_sil]
  database_input = database_input[database_input.dbs<=max_dbs]
  return database_input

def filter_dist_sil_bds(database_input, dist):
  database_input = database_input[(database_input.dbs-database_input.sil) <= dist]
  return database_input

def filter_samples_isolation(database_input, contamination):
    if contamination>0:
        iso = IsolationForest(contamination=contamination, random_state=39)
        yhat = iso.fit_predict(database_input._get_numeric_data())

        mask = yhat != -1
        database_input = database_input.iloc[mask, :]
    return database_input

def plot_train(model):
  results = model.evals_result()

  plt.figure(figsize=(10,7))
  plt.plot(results["validation_0"]["rmse"], label="Training loss")
  plt.plot(results["validation_1"]["rmse"], label="Validation loss")
  plt.axvline(model.best_ntree_limit, color="gray", label="Optimal tree number")
  plt.xlabel("Number of trees")
  plt.ylabel("Loss")
  plt.legend()

def model_prediction_analysis(model_regressor, dataset_test, dataset_y_test):
  print(dataset_test.shape)
  yhat = model_regressor.predict(dataset_test)

  mae = metrics.mean_absolute_error(dataset_y_test, yhat)
  print('MAE', mae)
  mse = metrics.mean_squared_error(dataset_y_test, yhat)
  print('MSE', mse)
  rmse = np.sqrt(mse) # or mse**(0.5)
  print('RMSE', rmse)
  print('Obtained:',yhat[0:10])
  r2 = metrics.r2_score(dataset_y_test,yhat)
  print('R2', r2)

def plot_optimization(x, best_k):
  best_k = best_k.reset_index()
  for i in x.Dataset.unique():
    plt.figure()
    sns.lineplot(data=x[x.Dataset == i], x="k_candidate", y="yhat")
    sns.lineplot(data=x[x.Dataset == i], x="k_candidate", y="sil")
    ax = sns.lineplot(data=x[x.Dataset == i], x="k_candidate", y="dbs")
    ax.axvline(x = x[x.Dataset == i].k_expected.unique(), ymin = 0, ymax = 1)
    ax.axvline(x = best_k[best_k.Dataset_x == i].k_candidate.unique(), ymin = 0, ymax = 1, c='r', linestyle='--')
    ax.set_title(i)

def minimizing(model_regressor, features, plot, log, seed):
  datasets = run_exp(model_regressor, features, datasets_selected_benchmarking)
  x = pd.DataFrame(datasets)
  x.columns = ["Dataset", "Algorithm", "sil", "dbs", "k_candidate", "k_expected", "yhat"]
  x['yhat'] = x.apply(lambda row: row.yhat[0], axis=1)
  x

  min_data = pd.DataFrame(x.groupby(["Dataset"])["yhat"].min()).reset_index()
  result = pd.merge(x, min_data, on="yhat")
  result

  mae = np.round(metrics.mean_absolute_error(result.k_candidate, result.k_expected),2)
  print("MAE",mae)
  mse = np.round(metrics.mean_squared_error(result.k_candidate, result.k_expected),2)
  print("MSE",mse)
  rmse = np.round(np.sqrt(mse),2) # or mse**(0.5)
  print("RMSE",rmse)
  r2 = np.round(metrics.r2_score(result.k_candidate,result.k_expected),2)
  print("R2",r2)
  acc = np.round(metrics.accuracy_score(result.k_candidate,result.k_expected),2)
  print("ACC",acc)

  print("Std Deviation", result.yhat.std())

  #print(result.groupby(['Dataset_x', 'Algorithm'])['k_candidate', 'k_expected'].mean())
  print(result.groupby(['Dataset_x'])['k_candidate', 'k_expected'].min())

  if plot:
    plot_optimization(x, result.groupby(['Dataset_x'])['k_candidate', 'k_expected'].mean())

  if log:

    log = [mae, mse, rmse, r2, acc, seed, type(model_regressor).__name__]
    log = pd.DataFrame(log).T
    log.columns = ["MAE", "MSE", "RMSE", "R2", "ACC", "seed", "algorithm"]
    log.to_csv(PATH+"log_05_07_2023.csv", mode='a', index=False, header=False)

def minimizing_logging(model_regressor, features, sil_ini, dbs_max, dist_s_d, seed, dist, qtd, run, progress, contamin, qtd_arvores):
  datasets = run_exp(model_regressor, features, datasets_selected_benchmarking)
  x = pd.DataFrame(datasets)
  x.columns = ["Dataset", "Algorithm", "sil", "dbs", "k_candidate", "k_expected", "yhat"]
  x['yhat'] = x.apply(lambda row: row.yhat[0], axis=1)
  x

  min_data = pd.DataFrame(x.groupby(["Dataset"])["yhat"].min().reset_index())
  result = pd.merge(x, min_data, on="yhat").drop_duplicates(subset='Dataset_x', keep="first")

  #print(result)

  mae = np.round(metrics.mean_absolute_error(result.k_candidate, result.k_expected),2)
  print("MAE",mae)
  mse = np.round(metrics.mean_squared_error(result.k_candidate, result.k_expected),2)
  print("MSE",mse)
  rmse = np.round(np.sqrt(mse),2) # or mse**(0.5)
  print("RMSE",rmse)
  r2 = np.round(metrics.r2_score(result.k_candidate,result.k_expected),2)
  print("R2",r2)
  acc = np.round(metrics.accuracy_score(result.k_candidate,result.k_expected),2)
  print("ACC",acc)

  std =  np.round(result.yhat.std(),2)
  print("Std Deviation", std)

  contamination = np.round(contamin,2)

  #print(result.groupby(['Dataset_x', 'Algorithm'])['k_candidate', 'k_expected'].mean())
  #print(result.groupby(['Dataset_x'])['k_candidate', 'k_expected'].min().reset_index())
  print(result.groupby(['Dataset_x'])[['k_candidate', 'k_expected']]
        .mean())

  log = [mae, mse, rmse, r2, acc, std, type(model_regressor).__name__, seed, sil_ini, dbs_max, dist_s_d, dist, qtd, contamination, qtd_arvores]
  log = pd.DataFrame(log).T
  log.columns = ["MAE", "MSE", "RMSE", "R2", "ACC",  "STD all clusterers", "algorithm", "seed",  "sil_ini", "dbs_max", "distance_s_d", "distribution", "qtd samples", "contamination", "qtd_arvores"]
  log.to_csv(PATH+LOG_FILE, mode='a', index=False, header=False)

  run.log({"model_regressor": "RandomForest",
             "features": features,
             "sil_ini": sil_ini,  # time cost
             "dbs_max": dbs_max,  # parsimonly?
             "dist_s_d": dist_s_d,
             "seed": seed,
             "Distribution": dist,
             "Qtd Samples": qtd,
             "MAE": mae,
             "MSE": mse,
             "RMSE": rmse,
             "R2": r2,
             "ACC": acc,
             "STD all clusterers": std,
             "Progress":progress,
           "Contamination": contamination,
           "qtd_arvores":qtd_arvores
             })

"""## benchmarking datasets and features"""

datasets_selected_benchmarking = [
                    'a1.csv',
                    'iris.csv',
                    'dim1024.csv', 'dim4.csv', 'dim5.csv', 'dim15.csv',
                    'dim7.csv', 'dim6.csv',# 'dim8.csv', 'dim9.csv',
                    #'dim512.csv', 'dim13.csv', 'dim10.csv', 'dim032.csv', 'dim12.csv', 'dim11.csv', 'dim14.csv',
                    #'dim064.csv', 'dim256.csv',
                    'dim128.csv', 'dim3.csv',
                    'chainlink.csv',
                    'cancer.csv',
                    #'a2.csv',
                    #'a3.csv',
                    'atom.csv', 'lsun3d.csv', 'hepta.csv', 'engytime.csv',
                    'target.csv', 'tetra.csv', 'twodiamonds.csv'#, 'wingnut.csv'
                    ,'unbalance.csv'
                    ]

features = ['attr_conc.mean','attr_conc.sd','attr_ent.mean',
            'attr_ent.sd','attr_to_inst','cohesiveness.mean','cohesiveness.sd',
            'cor.mean',#'cor.sd',
            'cov.mean',#'cov.sd',
            'eigenvalues.mean','eigenvalues.sd',
            'inst_to_attr','iq_range.mean','iq_range.sd','kurtosis.mean','kurtosis.sd',
            'mad.mean','mad.sd',
            #'max.mean','max.sd','mean.mean','mean.sd',
            'median.mean',
            'median.sd','min.mean','min.sd','nr_attr','nr_cor_attr','nr_inst','one_itemset.mean',
            'one_itemset.sd',
            #'range.mean','range.sd',
            'sd.mean','sd.sd','skewness.mean',
            'skewness.sd','sparsity.mean','sparsity.sd','t2','t3','t4','t_mean.mean',
            't_mean.sd','two_itemset.mean','two_itemset.sd','var.mean','var.sd',
            'wg_dist.mean','wg_dist.sd',
            'sil', 'dbs', 'clusters', 'cluster_diff'
       ]

features_4bench = ['attr_conc.mean','attr_conc.sd','attr_ent.mean',
    'attr_ent.sd','attr_to_inst','cohesiveness.mean','cohesiveness.sd',
    #'cor.mean',#'cor.sd',
    'cov.mean',#'cov.sd',
    'eigenvalues.mean','eigenvalues.sd',
    'inst_to_attr','iq_range.mean','iq_range.sd',
    #'kurtosis.mean','kurtosis.sd',
    'mad.mean','mad.sd',
    #'max.mean','max.sd','mean.mean','mean.sd',
    'median.mean',
    'median.sd',#'min.mean','min.sd',
    'nr_attr','nr_cor_attr','nr_inst','one_itemset.mean',
    'one_itemset.sd',
    #'range.mean','range.sd',
    'sd.mean','sd.sd'
    #,'skewness.mean', 'skewness.sd'
    ,'sparsity.mean','sparsity.sd','t2','t3','t4','t_mean.mean',
    't_mean.sd','two_itemset.mean','two_itemset.sd','var.mean','var.sd',
    'wg_dist.mean','wg_dist.sd',
    'sil', 'dbs', 'clusters', 'cluster_diff'
]

features = features_4bench

"""## run optimization"""


def run_exp(model_input, features, datasets_selected_benchmarking):

  features_local = features[:-4]

  #model = xgb.XGBRegressor()
  #model.load_model(PATH+"/xgb_Sv5.json")

  k_set = range(2,25,1)
  #print(k_set)

  mypath = PATH+"validation_set_small"
  onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

  df_benchmark = pd.read_csv(PATH+"metadatabase_validation_scaled.csv")
  df_benchmark = df_benchmark.iloc[::4,:]

  #datasets_selected = ['iris.csv','a1.csv','tetra.csv','hepta.csv', 'target.csv', 'dim3.csv',
  #                     'dim21-clusters10-instances2250-overlap1e-06-1e-05-aspectref3-aspectmaxmin3-radius5-imbalance1-rep1.csv',
  #                     'dim25-clusters2-instances200-overlap1e-06-1e-05-aspectref1.5-aspectmaxmin1-radius1-imbalance1-rep1.csv',
  #                     'dim6-clusters12-instances450-overlap1e-06-1e-05-aspectref1.5-aspectmaxmin3-radius1-imbalance1-rep1.csv']

  #datasets_selected = ['dim15.csv','a1.csv','dim9.csv','dim12.csv','dim11.csv','wingnut.csv','unbalance.csv'
#,'twodiamonds.csv','target.csv','tetra.csv','lsun3d.csv','iris.csv'
#,'dim256.csv','dim128.csv','dim064.csv','dim032.csv','dim14.csv'
#,'hepta.csv','dim13.csv','engytime.csv','dim10.csv','dim6.csv','dim5.csv'
#,'dim3.csv','chainlink.csv','cancer.csv','dim4.csv','atom.csv'
#,'dim7.csv','dim1024.csv','dim8.csv','dim512.csv']


  datasets_selected = datasets_selected_benchmarking

  # Extract all available unsupervised measures
  print("Datasets: ",datasets_selected)
  datasets = []
  for file in datasets_selected:

    #print(">>>"+file+"<<<<")
    #try:
    n_clusters_ideal = int(df_benchmark[df_benchmark["Dataset"] == file].clusters)
    #except:
    #  print("ERROR**********************")
    #  continue
    #print("n_clusters", n_clusters_ideal)

    X = pd.read_csv(mypath+"/"+file)
    ft = df_benchmark[df_benchmark.Dataset == file].filter(features_local)
    ft = ft[features_local].values

    #print(X.isnull().any().head(40))

    for rep in k_set:
      cluster_algo = [AgglomerativeClustering(n_clusters=rep, metric='euclidean', linkage='ward'),
                      KMeans(n_clusters=rep, n_init="auto"),
                      KMedoids(n_clusters=rep),
                      MiniBatchKMeans(n_clusters=rep, batch_size=10,n_init="auto")
                      ]
      sample_test = []
      for c in cluster_algo:
        cluster_labels = c.fit_predict(X)
        sil = silhouette_score(X, cluster_labels)
        dbs = davies_bouldin_score (X, cluster_labels)
        mf = ft[0].tolist()
        mf.extend([sil,dbs,rep])
        yhat = model_input.predict([mf])
        datasets.append([file]+[type(c).__name__]+[sil]+[dbs]+[rep]+[n_clusters_ideal]+[yhat])
  return datasets

"""# GridSearch"""



original = pd.read_csv(PATH+FILE)

#combined_lists = [sil_ini_list, dbs_max_list, dist_sil_dbs_list, seed_list, distribution_list, algorithms]
combined_lists = [sil_ini_list, dbs_max_list, dist_sil_dbs_list, seed_list, distribution_list, algorithms, contamination, qtd_arvores]

all_combinations = list(itertools.product(*combined_lists))

for index, combination in enumerate(all_combinations):
  #run.update()
  df_surrogate = original
  progress = (index + 1) / len(all_combinations) * 100
  print("\n\n\n")
  print(f"Progress: {progress:.2f}%")
  sil_ini, dbs_max, dist_s_d, seed, dist, regressor, contamin, qtd_arvores = combination

  #df_surrogate = df_surrogate.sample(frac=1, replace=True, random_state=1).reset_index(drop=True)
  #df_surrogate = df_surrogate.drop_duplicates(subset=["Dataset"], keep="last")

  df_surrogate = filtering_distribution(df_surrogate, dist)
  df_surrogate = filter_sil_bds(df_surrogate, sil_ini, dbs_max)
  df_surrogate = filter_dist_sil_bds(df_surrogate, dist_s_d)

  df_surrogate = filter_samples_isolation(df_surrogate, contamin)


  if(df_surrogate.shape[0]<threshold_qtd_on_training):
    print("Invalid Combination: ",df_surrogate.shape )
    continue
  print("Samples", df_surrogate.shape)

  df_surrogate['clusters'] = df_surrogate.apply(lambda row: int(row.Dataset.split('-')[1].replace("clusters","")), axis=1)

  df_surrogate = df_surrogate[features]

  data = df_surrogate
  x_train, y_train = data.values[:, :-1], data.values[:, -1]

  model_regressor = regressor(random_state=seed, n_estimators=qtd_arvores, n_jobs=-1)
  model_regressor.fit(x_train, y_train)

  run = wandb.init(project="Surrogate_Sv5", entity="barbonjr", reinit=True, name="A_"+str(round(contamin,2))+"_"+str(seed)+"_"+str(round(sil_ini,2))+"_"+str(round(dbs_max,2))+"_"+str(round(dist_s_d,2)))

  minimizing_logging(model_regressor, features, sil_ini, dbs_max, dist_s_d, seed, dist, df_surrogate.shape[0], run, progress, contamin, qtd_arvores)

  ### WANDB
  #run.log({"Progress": progress})
  run.finish()
  ### WANDB

"""#Persistindo o Surrogate"""

# export surrogate model
#model.save_model(PATH+"xgb_"+VERSION+".json")

Index(['file_name', 'attr_conc.mean', 'attr_conc.sd', 'attr_ent.mean',
       'attr_ent.sd', 'attr_to_inst', 'cohesiveness.mean', 'cohesiveness.sd',
       'cor.mean', 'cor.sd', 'cov.mean', 'cov.sd', 'eigenvalues.mean',
       'eigenvalues.sd', 'inst_to_attr', 'iq_range.mean', 'iq_range.sd',
       'kurtosis.mean', 'kurtosis.sd', 'mad.mean', 'mad.sd', 'max.mean',
       'max.sd', 'mean.mean', 'mean.sd', 'median.mean', 'median.sd',
       'min.mean', 'min.sd', 'nr_attr', 'nr_cor_attr', 'nr_inst',
       'one_itemset.mean', 'one_itemset.sd', 'range.mean', 'range.sd',
       'sd.mean', 'sd.sd', 'skewness.mean', 'skewness.sd', 'sparsity.mean',
       'sparsity.sd', 't2', 't3', 't4', 't_mean.mean', 't_mean.sd',
       'two_itemset.mean', 'two_itemset.sd', 'var.mean', 'var.sd',
       'wg_dist.mean', 'wg_dist.sd'],
      dtype='object')